In [1]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.4.1


In [2]:
lv1_08 = pd.read_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\2009_lv1_08.csv',index_col='Unnamed: 0')
Temperature_08 = pd.read_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\2009_Temperature_08.csv',index_col='Unnamed: 0')
merge_dataset =pd.merge(lv1_08,Temperature_08,on=lv1_08.index)
merge_dataset = merge_dataset.set_index(['key_0'])
merge_dataset.dropna(inplace=True)

In [3]:
# 乱序，分离训练数据与测试数据
train_data = merge_dataset.sample(frac=0.8,random_state=0)
test_data = merge_dataset.drop(train_data.index)
# 分离标签
train_labels = train_data.iloc[:,21:]
test_labels = test_data.iloc[:,21:]
train_data.drop(train_labels.columns,axis=1,inplace=True)
test_data.drop(train_labels.columns,axis=1,inplace=True)

print(train_data.shape,test_data.shape,train_labels.shape,test_labels.shape)

(249, 21) (62, 21) (249, 47) (62, 47)


In [4]:
train_stats = merge_dataset.describe()
train_stats = train_stats.transpose()
train_stats.drop(train_stats.iloc[20:].index,inplace=True)

In [5]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_data)
normed_test_data = norm(test_data)

In [6]:
print(train_data.columns,normed_train_data.columns)

Index(['10', 'Tamb(K)', 'Rh(%)', 'Pres(mb)', 'Tir(K)', 'Rain', 'Azim', 'Elev',
       'TkBB(K)', ' 22.235', ' 23.035', ' 23.835', ' 26.235', ' 30.000',
       ' 51.250', ' 52.280', ' 53.850', ' 54.940', ' 56.660', ' 57.290',
       ' 58.800'],
      dtype='object') Index([' 22.235', ' 23.035', ' 23.835', ' 26.235', ' 30.000', ' 51.250',
       ' 52.280', ' 53.850', ' 54.940', ' 56.660', ' 57.290', ' 58.800', '10',
       'Azim', 'Elev', 'Pres(mb)', 'Rain', 'Rh(%)', 'Tamb(K)', 'Tir(K)',
       'TkBB(K)'],
      dtype='object')


In [7]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(49)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [8]:
model = build_model()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 49)                3185      
Total params: 8,753
Trainable params: 8,753
Non-trainable params: 0
_________________________________________________________________
